## Install and Import Essential Packages To Setup Models and API Endpoints

In [ ]:
# Model
!pip install -q transformers accelerate einops langchain bitsandbytes cohere tiktoken
!pip install -q --upgrade openai
!pip install -q --upgrade langchain

# For API
!pip -q install fastapi
!pip -q install pyngrok
!pip -q install uvicorn
!pip -q install nest_asyncio

In [ ]:
# Mount google drive for data persistence (you can usse other storage options if you prefer).
from google.colab import drive

drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
import pandas as pd
import os

import openai
import os
import IPython
from langchain.llms import OpenAI

In [ ]:
# Provide your OpenAI API Key
os.environ['OPENAI_API_KEY'] = '' # Replace with your OpenAI API Key, or leave empty if you don't have one

In [ ]:
import uvicorn
import fastapi
from pyngrok import ngrok
from pydantic import BaseModel
import nest_asyncio

nest_asyncio.apply()

## Mistral-7B-Instruct Setup

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
# Fetch Mistral-7B-Instruct-v0.1 model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=500,
        temperature = 1,
        do_sample=False,
        #top_k=1,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        torch_dtype=torch.float16,
)

## Mistral Setup For Live And Demo Question (Riddle) Answering

In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

# Function to get mistral answer for live riddles
def answer_with_mistral(riddle):
  template = """  <s>[INST]   You are a science prodigy currently competing in a National Science competition. You are now in the fifth round, where you must first reason through the clues of the given riddle and then provide a short answer. Remember, your answer should consist of just the term the riddle is pointing to, and nothing else. Adding additional text will result in point deductions.
      Here's an example to guide you:
      Riddle: You might think i am a rather unstable character because i never stay at one place. However my motion obeys strict rules and i always return to where i started and even if i have to leave that spot again i do it in strict accordance to time. I can be named in electrical and mechanical contexts in all cases i obey the same mathematical rules. In order to fully analyse me you would think about a stiffness or force constant restoring force and angular frequency.
      Answer: oscillator

      Read the riddle below and provide the three possible correct answers as a json with keys: answer1, answer2, answer3

      NOTE: You are allowed to include an answer multiple times if your reasoning shows that it is likely the correct answer. Do not provide any explanations.

      Riddle: {riddle}

      [/INST] </s>

  """

  prompt = PromptTemplate(template=template, input_variables=["riddle"])
  llm_chain = LLMChain(prompt=prompt, llm=llm)
  answer = llm_chain.run({"riddle":riddle})
  return answer

In [ ]:
demo_llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"temperature": 0.0})

# Function to get mistral answer for demo riddles
def demo_qa_mistral_answer(riddle_content):
  template = """ <s>[INST] You are a science prodigy currently competing in a National Science competition. You are now in the fifth round, where you must provide a short answer to a riddle. Remember, your answer should consist of just the term the riddle is pointing to, and nothing else. Adding additional text will result in point deductions.
      Here's an example to guide you:
      Riddle: you might think i am a rather unstable character because i never stay at one place, however my motion obeys strict rules and i always return to where i started and even if i have to leave that spot again i do it in strict accordance to time, i can be named in electrical and mechanical contexts in all cases i obey the same mathematical rules, in order to fully analyse me you would think about a stiffness or force constant restoring force and angular frequency,
      Answer: oscillator

      Read the riddle below and provide the correct answer.

     Riddle: {riddle}

      [/INST] </s>
  """

  prompt = PromptTemplate(template=template, input_variables=["riddle"])
  falcon_chain = LLMChain(prompt=prompt, llm=demo_llm)
  answer = falcon_chain.run({"riddle":riddle_content})
  return answer.strip()

## ChatGPT Setup For Live Question (Riddle) Answering

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# chat mode instance
chat = ChatOpenAI(
    temperature=1.0)


# Function to get ChatGPT answer for live riddle
def live_qa_chatgpt_answer(riddle):
    template = """You are a science prodigy currently competing in a National Science competition. You are now in the fifth round, where you must first reason through the clues of the given riddle and then provide a short answer. Remember, your answer should consist of just the term the riddle is pointing to, and nothing else. Adding additional text will result in point deductions.
      Here's an example to guide you:
      Riddle: You might think i am a rather unstable character because i never stay at one place. However my motion obeys strict rules and i always return to where i started and even if i have to leave that spot again i do it in strict accordance to time. I can be named in electrical and mechanical contexts in all cases i obey the same mathematical rules. In order to fully analyse me you would think about a stiffness or force constant restoring force and angular frequency.
      Answer: oscillator

      Read the riddle below and provide the three possible correct answers as a json with keys: answer1, answer2, answer3

      NOTE: You are allowed to include an answer multiple times if your reasoning shows that it is likely the correct answer. Do not provide any explanations.

      Riddle: {riddle}

    """

    answer = chat([HumanMessage(content=template.format(riddle=riddle))])
    return answer.content

## Utility Functions And Confidence Modelling Function For Generated Answers

In [ ]:
import os
import csv
import json
import random
from glob import glob
import time
import string
import re

# Bring log directory from google drive into scope
LOG_DIR = "/content/drive/My Drive/NSMQ AI Project/Competition Logs/QA_LOGS"  # REPLACE WITH YOUR DESIRED DIRECTORY
RIDDLE_ANSWERED_FILE_LIVE = os.path.join(LOG_DIR, "ans_live_logs.json")
with open(RIDDLE_ANSWERED_FILE_LIVE, 'w') as f:
    json.dump({"Mistral": '', "ChatGPT": ''}, f)

def remove_articles(text):
    """
        Remove articles [the|a|an] from `text`

        Args:
            text: str

        Returns:
            text with articles removed: str
    """
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

def normalize_text(s):
    """
        Removing articles and punctuation, and standardizing whitespace are all typical text processing steps.

        Args:
            s: (str) string to normalize

        Returns:
            normalized string: str
    """

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation.replace("/", ""))
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))



def model_answer_confidence(model, threshold, chunk_num, model_answer, is_start_of_riddle):
    """
        Calculate confidence scores for model-generated answers.

        Args:
            model_name (str): The name of the language model.
            confidence_threshold (float): The confidence score threshold for accepting answers.
            chunk_num (int): The chunk number for the riddle.
            model_output (list): A list of answers generated by the model.
            is_start_of_riddle (bool): Indicates if this is the start of a new riddle.

        Returns:
            tuple: A tuple containing a list of answers and their associated confidence scores.

        This function calculates confidence scores for answers generated by a language model
        and returns a tuple of answers and their confidence scores.
    """
    cur_time = time.strftime("%Y-%m-%d_%H-%M-%S")  # Get the current time

    if is_start_of_riddle or chunk_num == 1:
        # Create a new JSON log file if it is a new riddle.
        print(f"Is Start of Riddle: {is_start_of_riddle}\t Clue Count: {chunk_num}")
        filename = os.path.join(LOG_DIR, f"{model}_log_{cur_time}.json")
        answer_counts = {}
    else:
        # Find the most recent generated JSON log file, if it is not a new riddle.
        log_files = glob(os.path.join(LOG_DIR, f"{model}_log_*.json"))
        if log_files:
            # Sort the log files by modification time to get the most recent one.
            log_files.sort(key=os.path.getmtime, reverse=True)
            filename = log_files[0]
            with open(filename, 'r') as f:
                logged_data = json.load(f)
                answer_counts = logged_data["answer_counts"]
                print("Loaded Answer Counts Dictionary:", answer_counts)
        else:
            # If no log files exist, create a new one.
            filename = os.path.join(LOG_DIR, f"{model}_log_{cur_time}.json")
            answer_counts = {}

    # Update answer_counts based on the model answer and chunk number
    for ans in model_answer:
        ans = remove_articles(normalize_text(ans).replace('"', '')).strip()
        answer_counts[ans] = answer_counts.get(ans, 0) + int(chunk_num)

    answer_counts[''] = 0

    print("Answer Counts Dictionary:", answer_counts)

    # Find the top answers and write answer_counts to the JSON log file
    top_count = max(answer_counts.values())
    top_answers = [ans for ans, count in answer_counts.items() if count == top_count]

    top_answer = ""

    if top_count >= threshold:
        top_answer = random.choice(top_answers)

    with open(filename, 'w') as f:
        data_to_save = {
            "answer_counts": answer_counts,
            "top_answer": (top_answer, top_count)
        }
        json.dump(data_to_save, f)

        print("Saved Data:", data_to_save)

    return top_answer, top_count


In [ ]:
import ast
import json


def preprocess_model_output(model_output):
    """
      Preprocess the output from a language model.

      Args:
          model_output (dict): The raw output from a language model.

      Returns:
          dict: The preprocessed model output.

      This function performs any necessary preprocessing on the model's output
      to convert the model's output into a json for the next steps.
    """
    if isinstance(model_output, dict):
        return model_output

    # Convert model output to string
    model_output = str(model_output).replace("\n", '').strip()

    # Remove all text that are not enclosed in '{' and '}'
    pattern = r'{.*?}'
    m = re.search(pattern, model_output)
    model_output = m.group(0)

    # Remove ` characters if any
    model_output = model_output.replace('```', '').replace('json', '')

    # Surround model output in curly braces if it isn't already.
    if not model_output.startswith("{") or not model_output.endswith("}"):
        model_output = '{' + model_output + '}'

    # Replace null in quotes and replace with none
    model_output = model_output.replace(": null", "'null'")
    print("State of Model Output:", model_output)

    # Try converting answer to json
    try:
        json_data = json.loads(model_output)
        return json_data
    except (SyntaxError, ValueError):
        print("SOMETHING WENT WRONG!")
        return None


## FastAPI Endpoints for Live and Demo Question-Answering

In [ ]:
class DemoInputText(BaseModel):
    """
        Data model for demo input containing text.

        Attributes:
            text (str): The text content for the demo input.
    """
    text: str


class LiveInputText(BaseModel):
    """
        Data model for live input containing clues and metadata.

        Attributes:
            clues (str): The clues or questions for the live input.
            is_start_of_riddle (bool): Indicates if this is the start of a new riddle.
            is_end_of_riddle (bool): Indicates if this is the end of a riddle.
            clue_count (int): The count of clues provided.
    """
    clues: str
    is_start_of_riddle: bool = False
    is_end_of_riddle: bool = False
    clue_count: int = 0


class LiveDemoInputText(BaseModel):
    """
        Data model for live demo input containing clues, metadata, and a threshold.

        Attributes:
            clues (str): The clues or questions for the live demo input.
            is_start_of_riddle (bool): Indicates if this is the start of a new riddle.
            is_end_of_riddle (bool): Indicates if this is the end of a riddle.
            clue_count (int): The count of clues provided.
            threshold (int): The threshold value, set to 4 by default.
    """
    clues: str
    is_start_of_riddle: bool = False
    is_end_of_riddle: bool = False
    clue_count: int = 0
    threshold = 4

class OutputText(BaseModel):
    """
        Data model for output text, including Mistral and ChatGPT responses.

        Attributes:
            mistral (str): The response from the Mistral model.
            chatGPT (str, optional): The response from the ChatGPT model, which is optional.
    """
    mistral: str
    chatGPT: str = None

In [ ]:
app = fastapi.FastAPI()


def filter_answers(ans_data, confidence_threshold, is_end_of_riddle):
    """
        Filter answers based on confidence threshold and riddle completion status.

        Args:
            ans_data (tuple): A tuple containing answer data, where the first element is the answer
                and the second element is the confidence score.
            confidence_threshold (float): The confidence score threshold for accepting answers.
            is_end_of_riddle (bool): Indicates whether this is the end of the riddle.

        Returns:
            str: The filtered answer if the confidence score is above the threshold or if it's
            the end of the riddle; otherwise, an empty string.
    """
    confidence = ans_data[1]
    if int(confidence) >= confidence_threshold or is_end_of_riddle:
      return ans_data[0]

    return ''


def load_riddle_answered_log(is_start_of_riddle=True, chunk_num=1):
    """
        Load previously answered riddle data from a file.

        Args:
            is_start_of_riddle (bool): Indicates if this is the start of a new riddle.
            chunk_num (int): The chunk number for the riddle.

        Returns:
            dict: A dictionary containing previously answered riddles, with keys 'mistral' and 'chatGPT'.
            If it's the start of a riddle, an empty dictionary is returned.
    """
    if is_start_of_riddle == True:
      return {"mistral": '' , "chatGPT": ''}
    else:
      if os.path.exists(RIDDLE_ANSWERED_FILE_LIVE):
        with open(RIDDLE_ANSWERED_FILE_LIVE, "r") as file:
          return json.load(file)
      else:
        return {"mistral": '', "chatGPT": ''}

def save_riddle_answered_log(data):
    """
        Save riddle answer data to a file.

        Args:
            data (dict): A dictionary containing riddle answer data to be saved.

        This function saves the answer data to a file for later retrieval.
    """
    with open(RIDDLE_ANSWERED_FILE_LIVE, "w") as file:
        json.dump(data, file)


@app.get("/live_qa", response_model=OutputText)
def live_answer(input_data: LiveInputText):
    """
        Perform live answering for a given input containing clues and metadata.

        Args:
            input_data (LiveInputText): Input data containing clues and metadata (is_start_of_riddle, chunk_num and is_end_of_riddle).

        Returns:
            dict: A dictionary containing answers generated using Mistral and ChatGPT.
                Keys are 'mistral' and 'chatGPT', and values are lists of answers.

        This function loads previously computed answers, uses the Mistral model to generate answers
        if necessary, and filters and stores the results. It returns a dictionary with the answers.
    """
    ct = 10.0  # ct represents the confidence threshold.
    chunk_num = input_data.clue_count
    is_start_of_riddle = input_data.is_start_of_riddle
    is_end_of_riddle = input_data.is_end_of_riddle


    # Load data previously computed answers
    answer_file = load_riddle_answered_log(is_start_of_riddle, chunk_num)

    if answer_file['mistral'] == '' and chunk_num != 0:
        # Send clues to mistral-7b and get answer if we haven't answered the riddle yet
        mistral_output = answer_with_mistral(riddle=input_data.clues)
        mistral_output = preprocess_model_output(mistral_output)

        #Put answers in a list
        if mistral_output is not None:
            mistral_output = [mistral_output[key] for key in mistral_output.keys()]
        else:
          mistral_output = ['']
          print("Failed to convert Mistral response to dict/json")
        mistral_ans_data = model_answer_confidence("Mistral", ct, chunk_num, mistral_output, is_start_of_riddle)
        mistral_final_ans = filter_answers(mistral_ans_data, ct, is_end_of_riddle)
    else:
      # If we've already answered on previous clues, load that answer instead
      mistral_final_ans = answer_file['mistral']

    if answer_file['chatGPT'] == '' and chunk_num != 0:
        # Send clues to ChatGPT and get answer if we havent't answered the riddle yet
        if os.environ['OPENAI_API_KEY'] != '':
            chatGPT_output = live_qa_chatgpt_answer(input_data.clues)
        else:
            chatGPT_output = {"answer1": ''}
        chatGPT_output = preprocess_model_output(chatGPT_output)
        # Put answers in a list
        if chatGPT_output is not None:
            chatGPT_output = [chatGPT_output[key] for key in chatGPT_output.keys()]
        else:
          chatGPT_output = ['',]
          print("Failed to convert ChatGPT response to dict/json")
        chatGPT_ans_data = model_answer_confidence("ChatGPT", ct, chunk_num, chatGPT_output, is_start_of_riddle)
        chatgpt_final_ans = filter_answers(chatGPT_ans_data, ct, is_end_of_riddle)
    else:
      # If we've already answered on previous clues, load that answer instead
      chatgpt_final_ans = answer_file['chatGPT']


    answers = {
        "mistral": mistral_final_ans,
        "chatGPT": chatgpt_final_ans
    }

    # Save answers data to file. This is to ensure that once we've returned an answer
    # for a riddle, we no longer do inference for subsequent clues till the next riddle.
    save_riddle_answered_log(answers)

    return answers


@app.get("/live_demo_qa", response_model=OutputText)
def live_demo_answer(input_data: LiveDemoInputText):
    """
        Perform live answering for a given input containing clues and metadata.

        Args:
            input_data (LiveInputText): Input data containing clues and metadata (is_start_of_riddle, chunk_num and is_end_of_riddle).

        Returns:
            dict: A dictionary containing answers generated using Mistral and ChatGPT.
                Keys are 'mistral' and 'chatGPT', and values are lists of answers.

        This function loads previously computed answers, uses the Mistral model to generate answers
        if necessary, and filters and stores the results. It returns a dictionary with the answers.
    """
    ct = 10.0  # ct represents the confidence threshold.
    chunk_num = input_data.clue_count
    is_start_of_riddle = input_data.is_start_of_riddle
    is_end_of_riddle = input_data.is_end_of_riddle


    # Load data previously computed answers
    answer_file = load_riddle_answered_log(is_start_of_riddle, chunk_num)

    if answer_file['mistral'] == '' and chunk_num != 0:
        # Send clues to mistral-7b and get answer if we haven't answered the riddle yet
        mistral_output = answer_with_mistral(riddle=input_data.clues)
        mistral_output = preprocess_model_output(mistral_output)

        #Put answers in a list
        if mistral_output is not None:
            mistral_output = [mistral_output[key] for key in mistral_output.keys()]
        else:
          mistral_output = ['']
          print("Failed to convert Mistral response to dict/json")
        mistral_ans_data = model_answer_confidence("Mistral", ct, chunk_num, mistral_output, is_start_of_riddle)
        mistral_final_ans = filter_answers(mistral_ans_data, ct, is_end_of_riddle)
    else:
      # If we've already answered on previous clues, load that answer instead
      mistral_final_ans = answer_file['mistral']

    if answer_file['chatGPT'] == '' and chunk_num != 0:
        # Send clues to ChatGPT and get answer if we havent't answered the riddle yet
        if os.environ['OPENAI_API_KEY'] != '':
            chatGPT_output = live_qa_chatgpt_answer(input_data.clues)
        else:
            chatGPT_output = {"answer1": ''}
        chatGPT_output = preprocess_model_output(chatGPT_output)
        # Put answers in a list
        if chatGPT_output is not None:
            chatGPT_output = [chatGPT_output[key] for key in chatGPT_output.keys()]
        else:
          chatGPT_output = ['',]
          print("Failed to convert ChatGPT response to dict/json")
        chatGPT_ans_data = model_answer_confidence("ChatGPT", ct, chunk_num, chatGPT_output, is_start_of_riddle)
        chatgpt_final_ans = filter_answers(chatGPT_ans_data, ct, is_end_of_riddle)
    else:
      # If we've already answered on previous clues, load that answer instead
      chatgpt_final_ans = answer_file['chatGPT']


    answers = {
        "mistral": mistral_final_ans,
        "chatGPT": chatgpt_final_ans
    }

    # Save answers data to file. This is to ensure that once we've returned an answer
    # for a riddle, we no longer do inference for subsequent clues till the next riddle.
    save_riddle_answered_log(answers)

    return answers

@app.get('/demo_qa', response_model=OutputText)
def demo_answer(input_data: DemoInputText):
    """
        Perform a demo question-answering using the Mistral model.

        Args:
            input_data (DemoInputText): Input data containing the riddle content.

        Returns:
            dict: A dictionary containing answers generated using the Mistral model.

        This function takes the riddle content as input, sends it to the Mistral model, and returns
        the model's generated answers in a dictionary with the 'mistral' key.
    """
    riddle_content = input_data.text
    mistral_ans = demo_qa_mistral_answer(riddle_content)

    answers = {
        "mistral": mistral_ans
    }
    return answers


@app.get('/test_qa', response_model=OutputText)
def test():
    # Test that endpoint is up and running.
    return {"mistral": "Hello from Mistral!", "chatGPT": "Hello from ChatGPT!"}

## Creating A Public Tunnel for FastAPI App With Ngrok

In [ ]:
# Add your ngrok auth token
!ngrok config add-authtoken #[YOU NGROK AUTH TOKEN]

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
uvicorn.run(app, port=8000)